In [ ]:
// Load Kotlin Notebook libraries
%use kandy, dataframe

In [ ]:
import java.io.File

// Load input and output files
val inputFile = File("resources/FLStartSound.wav")
val processedFile = File("resources/FLStartSound_processed.wav")

### FFT Helpers

In [ ]:
// Load FFT helper library and import some commonly used packages
@file:DependsOn("org.quifft:quifft:0.1.1")
import org.quifft.QuiFFT
import org.quifft.output.*

In [ ]:
/** Returns the number of frequency bins in an [FFTResult] */
val FFTResult.numBins get(): Int = fftFrames.firstOrNull()?.bins?.size ?: 0

/** Finds and returns the maximum amplitude of each frequency in an FFTResult as a list of [FrequencyBin]s  */
fun FFTResult.binPeaks(): List<FrequencyBin> = List(numBins) {
    fftFrames.maxOfWith(compareBy(FrequencyBin::amplitude)) { frame -> frame.bins[it] }
}

/** Helper method for plotting a frequency spectrum */
fun List<FrequencyBin>.plot() = run {
    val frequency by column(map { it.frequency })
    val amplitude by column(map { it.amplitude })

    dataFrameOf(frequency, amplitude).plot {
        line {
            x(frequency) {
                scale = continuous(transform = Transformation.LOG10)
            }
            y(amplitude) {
                scale = continuous(-65.0..0.0)	
            }
            color(amplitude) {
                scale = continuous(range = Color.GREEN..Color.RED, domain = -65.0..0.0, transform = Transformation.SYMLOG)
            }
        }
        layout {
            size = 700 to 300
        }
    }
}

### Audio Helpers

In [ ]:
// Load audio helper library and import some commonly used packages
@file:DependsOn("com.soywiz.korge:korge-core-jvm:5.1.0")
import korlibs.audio.format.*
import korlibs.audio.sound.*

In [ ]:
import korlibs.io.util.length

class CustomIndexRangeList<T>(val indexRange: IntRange, init: (Int) -> T) : List<T> {
    private val innerList: MutableList<T> = MutableList(indexRange.length) { init(it.customizeIndex()) }

    override val size: Int by innerList::size

    private fun Int.customizeIndex(): Int = this + indexRange.start
    @Throws(IllegalArgumentException::class)
    private fun Int.normalizeIndex(): Int {
        if (this !in indexRange) throw IllegalArgumentException("Index $this is not within bounds $indexRange")
        return this - indexRange.start
    }

    override operator fun get(index: Int): T = innerList[index.normalizeIndex()]
    operator fun set(index: Int, value: T) { innerList[index.normalizeIndex()] = value }
    override fun isEmpty(): Boolean = innerList.isEmpty()
    override fun iterator(): Iterator<T> = innerList.iterator()
    override fun listIterator(): ListIterator<T> = innerList.listIterator()
    override fun listIterator(index: Int): ListIterator<T> = innerList.listIterator(index.normalizeIndex())
    override fun subList(fromIndex: Int, toIndex: Int): List<T> = innerList.subList(fromIndex.normalizeIndex(), toIndex.normalizeIndex())
    override fun lastIndexOf(element: T): Int = innerList.lastIndexOf(element).customizeIndex()
    override fun indexOf(element: T): Int = innerList.indexOf(element).customizeIndex()
    override fun containsAll(elements: Collection<T>): Boolean = innerList.containsAll(elements)
    override fun contains(element: T): Boolean = innerList.contains(element)
}

In [ ]:
import kotlinx.coroutines.runBlocking

fun AudioSamples.toMono() = AudioSamples(1, totalSamples).also { out ->
	for (sampleIndex in 0..<totalSamples) {
		out.setFloat(0, sampleIndex, (0..<channels).map { getFloat(it, sampleIndex) }.average().toFloat())
	}
}

fun AudioSamples.feed(
    feedbackBufferRange: IntRange = 0..0,
    feedforwardBufferRange: IntRange = 0..0,
    transform: (sample: Float, feedbackBuffer: CustomIndexRangeList<Float>, feedfowardBuffer: CustomIndexRangeList<Float>) -> Float,
) = AudioSamples(channels, totalSamples).also { out ->
    for (channel in 0..<channels) {
        for (sampleIndex in 0..<totalSamples) {
            val currentSample = getFloat(channel, sampleIndex)
            val feedbackBuffer = CustomIndexRangeList<Float>(feedbackBufferRange) { indexOffset ->
                (sampleIndex + indexOffset).takeIf { it in 0..<out.totalSamples }?.let { out.getFloat(channel, it) } ?: 0f
            }
            val feedforwardBuffer = CustomIndexRangeList<Float>(feedforwardBufferRange) { indexOffset ->
                (sampleIndex + indexOffset).takeIf { it in 0..<totalSamples }?.let { getFloat(channel, it) } ?: 0f
            }
            out.setFloat(channel, sampleIndex, transform(currentSample, feedbackBuffer, feedforwardBuffer))
        }
    }
}

fun File.applyEffects(outputFile: File, compute: AudioData.() -> AudioData) = runBlocking {
	var data = WAV.decode(readBytes())!!
	outputFile.writeBytes(WAV.encodeToByteArray(data.compute()))
}

### EQ Utils

In [ ]:
import korlibs.math.PI2F
import korlibs.math.PIF
import kotlin.properties.Delegates

/**
 * @see <span>Robert Bristow-Johnson's <a href="https://archive.ph/20121220231853/http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt">cookbook formulae for audio EQ biquad filter coefficients</a></span>
 */
fun biquadTransform(
    sample: Float,
    inSample1: Float, inSample2: Float,
    outSample1: Float, outSample2: Float,
    b0: Float, b1: Float, b2: Float,
    a0: Float, a1: Float, a2: Float,
): Float = (b0 / a0) * sample + (b1 / a0) * inSample1 + (b2 / a0) * inSample2 - (a1 / a0) * outSample1 - (a2 / a0) * outSample2

data class Coefficients(
	val b0: Float, val b1: Float, val b2: Float,
	val a0: Float, val a1: Float, val a2: Float,
) {
    companion object {
        val Zero = Coefficients(0f, 0f, 0f, 0f, 0f, 0f)
    }
}

interface EQFilter {
    var sampleRate: Int
    var coefficients: Coefficients
    
    fun calculateCoefficients(sampleRate: Int): Coefficients
}

fun EQFilter.compute(
    sampleRate: Int,
    sample: Float,
    inSample1: Float, inSample2: Float,
    outSample1: Float, outSample2: Float,
): Float {
    if (this.sampleRate != sampleRate) {
        this.sampleRate = sampleRate
        coefficients = calculateCoefficients(sampleRate)
    }
    val (b0, b1, b2, a0, a1, a2) = coefficients
    return biquadTransform(sample, inSample1, inSample2, outSample1, outSample2, b0, b1, b2, a0, a1, a2)
}

data class Peak(val frequency: Float, val bandwidth: Float, val gain: Float) : EQFilter {
    private val A: Float = 10f.pow(gain / 40)

    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 * sinh(ln(2f) / 2 * bandwidth * w0 / sinw0)
        return Coefficients(
			b0 =  1 + alpha * A,
			b1 = -2 * cosw0,
			b2 =  1 - alpha * A,
			a0 =  1 + alpha / A,
			a1 = -2 * cosw0,
			a2 =  1 - alpha / A,
        )
    }
}
data class Notch(val frequency: Float, val bandwidth: Float) : EQFilter {
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 * sinh(ln(2f) / 2 * bandwidth * w0 / sinw0)
        return Coefficients(
            b0 =  1f,
            b1 = -2 * cosw0,
            b2 =  1f,
            a0 =  1 + alpha,
            a1 = -2 * cosw0,
            a2 =  1 - alpha,
        )
    }
}
data class HighShelf(val frequency: Float, val shelfSlope: Float, val gain: Float) : EQFilter {
    private val A: Float = 10f.pow(gain / 40)

    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / 2 * sqrt((A + 1 / A) * (1 / shelfSlope - 1) + 2)
        val srqtAalpha2 = 2 * sqrt(A) * alpha
        return Coefficients(
            b0 =      A * ((A + 1) + (A - 1) * cosw0 + srqtAalpha2),
            b1 = -2 * A * ((A - 1) + (A + 1) * cosw0),
            b2 =      A * ((A + 1) + (A - 1) * cosw0 - srqtAalpha2),
            a0 =           (A + 1) - (A - 1) * cosw0 + srqtAalpha2,
            a1 =      2 * ((A - 1) - (A + 1) * cosw0),
            a2 =           (A + 1) - (A - 1) * cosw0 - srqtAalpha2,
        )
    }
}
data class LowShelf(val frequency: Float, val shelfSlope: Float, val gain: Float) : EQFilter {
    private val A: Float = 10f.pow(gain / 40)
    
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / 2 * sqrt((A + 1 / A) * (1 / shelfSlope - 1) + 2)
        val srqtAalpha2 = 2 * sqrt(A) * alpha
        return Coefficients(
            b0 =     A * ((A + 1) - (A - 1) * cosw0 + srqtAalpha2),
            b1 = 2 * A * ((A - 1) - (A + 1) * cosw0),
            b2 =     A * ((A + 1) - (A - 1) * cosw0 - srqtAalpha2),
            a0 =          (A + 1) + (A - 1) * cosw0 + srqtAalpha2,
            a1 =    -2 * ((A - 1) + (A + 1) * cosw0),
            a2 =          (A + 1) + (A - 1) * cosw0 - srqtAalpha2,
        )
    }
}
data class LowPass(val frequency: Float, val resonance: Float) : EQFilter {
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / (2 * resonance)
        return Coefficients(
            b0 = (1 - cosw0) / 2,
            b1 =  1 - cosw0,
            b2 = (1 - cosw0) / 2,
            a0 =  1 + alpha,
            a1 = -2 * cosw0,
            a2 =  1 - alpha,
        )
    }
}
data class HighPass(val frequency: Float, val resonance: Float) : EQFilter {
    override var sampleRate: Int = 0
    override var coefficients: Coefficients = Coefficients.Zero
    
    override fun calculateCoefficients(sampleRate: Int): Coefficients {
        val w0 = PI2F * frequency / sampleRate
        val cosw0 = cos(w0)
        val sinw0 = sin(w0)
        val alpha = sinw0 / (2 * resonance)
        return Coefficients(
            b0 = (1 + cosw0) / 2,
            b1 = -1 - cosw0,
            b2 = (1 + cosw0) / 2,
            a0 =  1 + alpha,
            a1 = -2 * cosw0,
            a2 =  1 - alpha,
        )
    }
}

fun AudioData.equalize(vararg filters: EQFilter): AudioData = AudioData(rate, 
	filters.fold(samples) { samples, filter ->
		samples.feed(-2..-1, -2..-1) { sample, feedbackBuffer, feedforwardBuffer ->
			filter.compute(rate, sample, feedforwardBuffer[-1], feedforwardBuffer[-2], feedbackBuffer[-1], feedbackBuffer[-2])
		}
	}
)

### Plot initial frequency spectrum

In [ ]:
QuiFFT(inputFile).fullFFT().binPeaks().plot()

### Apply EQ

In [ ]:
inputFile.applyEffects(processedFile) {
	equalize(
//		LowPass(1000f, 1f),
//		HighPass(2000f, 1f),
		Notch(500f, 10f),
//      Peak(1000f, .2f, 50f),
//      LowShelf(250f, 1f, -50f), 
//		HighShelf(10000f, 1f, 20f),
	)
}

### Plot filtered frequency spectrum

In [ ]:
QuiFFT(processedFile).fullFFT().binPeaks().plot()